In [ ]:
import pandas as pd
CSV_PATH = "../data/benchmark_results.csv"

In [37]:
b_df = pd.read_csv(CSV_PATH)
b_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 42 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   timestamp                       72 non-null     object 
 1   hostname                        72 non-null     object 
 2   platform                        72 non-null     object 
 3   system                          72 non-null     object 
 4   release                         72 non-null     int64  
 5   version                         72 non-null     object 
 6   machine                         72 non-null     object 
 7   processor                       72 non-null     object 
 8   cpu_count_logical               72 non-null     int64  
 9   cpu_count_physical              72 non-null     int64  
 10  cpu_freq_max                    72 non-null     float64
 11  cpu_freq_current                72 non-null     float64
 12  memory_total_gb                 72 non

In [38]:
# Removing fireduck for now because there is not data(yet)
fireduck_columns = [col for col in b_df.columns if 'fireducks_' in col]
b_df.drop(columns=fireduck_columns, inplace=True)

b_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 38 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   timestamp                    72 non-null     object 
 1   hostname                     72 non-null     object 
 2   platform                     72 non-null     object 
 3   system                       72 non-null     object 
 4   release                      72 non-null     int64  
 5   version                      72 non-null     object 
 6   machine                      72 non-null     object 
 7   processor                    72 non-null     object 
 8   cpu_count_logical            72 non-null     int64  
 9   cpu_count_physical           72 non-null     int64  
 10  cpu_freq_max                 72 non-null     float64
 11  cpu_freq_current             72 non-null     float64
 12  memory_total_gb              72 non-null     float64
 13  memory_available_gb   

In [39]:
# Focus on 10 million rows for uniform comparison
b_df = b_df[b_df["dataset_size"] == 10_000_000]
b_df.shape

(68, 38)

In [40]:
time_cols = [col for col in b_df.columns if col.endswith('_seconds')]
b_df[time_cols] = b_df[time_cols].round(2)
b_df.head(3)

,timestamp,hostname,platform,system,release,version,machine,processor,cpu_count_logical,cpu_count_physical,...,statistics_duckdb_seconds,complex_join_pandas_seconds,complex_join_modin_seconds,complex_join_polars_seconds,complex_join_duckdb_seconds,timeseries_pandas_seconds,timeseries_modin_seconds,timeseries_polars_seconds,timeseries_duckdb_seconds,script_name
0,2025-08-20T22:58:21.887566,TS-P350-01,Windows-11-10.0.26100-SP0,Windows,11,10.0.26100,AMD64,"Intel64 Family 6 Model 167 Stepping 1, Genuine...",16,8,...,1.03,37.76,29.70,2.78,10.72,25.64,10.68,3.35,1.01,benchmark.py
1,2025-08-21T15:09:54.202383,WL5022,Windows-11-10.0.26100-SP0,Windows,11,10.0.26100,AMD64,"Intel64 Family 6 Model 140 Stepping 1, Genuine...",8,4,...,4.18,75.80,62.93,6.49,22.18,48.58,23.70,8.04,4.26,benchmark.py
2,2025-08-21T18:10:27.497627,MSI,Windows-11-10.0.26100-SP0,Windows,11,10.0.26100,AMD64,"AMD64 Family 26 Model 36 Stepping 0, AuthenticAMD",20,10,...,0.99,34.04,25.34,1.77,7.44,21.02,14.11,4.07,0.77,benchmark.py


In [41]:
def rank_methods(df, operation_col_prefix):
    time_cols = [col for col in df.columns if col.startswith(operation_col_prefix) and col.endswith('_seconds')]
    rank_df = pd.DataFrame()
    rank_df['hostname'] = df['hostname']
    rank_df['cpu_brand'] = df['cpu_brand'] 
    rank_df['dataset_size'] = df['dataset_size']
    rank_df['dataset_format'] = df['dataset_format']
    
    for col in time_cols:
        rank_col = f"rank_{col}"
        # Only rank non-zero values, assign max rank to zeros
        mask = df[col] > 0
        rank_df[rank_col] = df[col].where(mask).rank(method='min')
        max_rank = rank_df[rank_col].max()
        rank_df[rank_col] = rank_df[rank_col].fillna(max_rank + 1)
    
    return rank_df

In [42]:
filter_group_ranks = rank_methods(b_df, 'filter_group_')
sort_values_cols = [col for col in filter_group_ranks.columns if col.startswith('rank_filter_group_')]
filter_group_ranks[sort_values_cols] = filter_group_ranks[sort_values_cols].astype(int)
filter_group_ranks = filter_group_ranks.sort_values(by=sort_values_cols + ['hostname', 'dataset_format'])
filter_group_ranks

,hostname,cpu_brand,dataset_size,dataset_format,rank_filter_group_pandas_seconds,rank_filter_group_modin_seconds,rank_filter_group_polars_seconds,rank_filter_group_duckdb_seconds
56,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,parquet,1,3,4,9
46,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,parquet,2,6,13,1
25,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,parquet,3,20,15,5
50,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,parquet,4,1,2,5
55,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,parquet,4,19,5,9
...,...,...,...,...,...,...,...,...
59,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,10000000,csv,64,45,55,52
12,IdeaPadS340,Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz,10000000,csv,65,62,57,61
9,IdeaPadS340,Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz,10000000,csv,66,65,56,57
19,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,10000000,csv,67,60,61,60


In [43]:
def get_df_by_test(df: pd.DataFrame, cols_to_group: list, metric: str) -> pd.DataFrame:
    # Properly combine columns - create a new list instead of using append
    columns_to_select = cols_to_group + [metric]
    
    # Select the relevant columns
    df_subset = df[columns_to_select]
    
    # Remove rows where the metric is null/zero (assuming you want valid measurements)
    df_filtered = df_subset[df_subset[metric] > 0]  # or use .notna() if you want non-null values
    
    # Group by the specified columns and get minimum values
    grouped = df_filtered.groupby(cols_to_group)
    return grouped.min().reset_index().sort_values(by=metric)

In [44]:
cols_to_group = [ "dataset_format","hostname", "cpu_brand"]
for col_to_rank in time_cols:
    print(f"Ranking by: {col_to_rank}")
    df_by_test = get_df_by_test(b_df, cols_to_group, col_to_rank)
    display(df_by_test)
    print("\n")

Ranking by: filter_group_pandas_seconds


,dataset_format,hostname,cpu_brand,filter_group_pandas_seconds
9,parquet,Inspiron3030S,Intel(R) Core(TM) i7-14700,4.54
10,parquet,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,5.03
12,parquet,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,5.36
13,parquet,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,6.10
11,parquet,Optiplex7010,13th Gen Intel(R) Core(TM) i9-13900,6.30
6,csv,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,10.41
2,csv,Inspiron3030S,Intel(R) Core(TM) i7-14700,10.67
3,csv,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,11.96
0,csv,HP-Desktop,12th Gen Intel(R) Core(TM) i9-12900,12.34
5,csv,Optiplex7010,13th Gen Intel(R) Core(TM) i9-13900,13.57




Ranking by: filter_group_modin_seconds


,dataset_format,hostname,cpu_brand,filter_group_modin_seconds
9,parquet,Inspiron3030S,Intel(R) Core(TM) i7-14700,5.79
2,csv,Inspiron3030S,Intel(R) Core(TM) i7-14700,6.50
10,parquet,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,6.67
0,csv,HP-Desktop,12th Gen Intel(R) Core(TM) i9-12900,7.18
12,parquet,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,7.46
6,csv,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,7.53
13,parquet,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,7.62
3,csv,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,8.30
4,csv,MSI,AMD Ryzen AI 9 365 w/ Radeon 880M,10.05
7,csv,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,10.72




Ranking by: filter_group_polars_seconds


...

,dataset_format,hostname,cpu_brand,timeseries_modin_seconds
1,csv,IdeaPadS340,Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz,3.00
9,parquet,Inspiron3030S,Intel(R) Core(TM) i7-14700,4.12
10,parquet,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,6.41
6,csv,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,7.31
2,csv,Inspiron3030S,Intel(R) Core(TM) i7-14700,7.39
0,csv,HP-Desktop,12th Gen Intel(R) Core(TM) i9-12900,7.69
13,parquet,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,8.17
4,csv,MSI,AMD Ryzen AI 9 365 w/ Radeon 880M,9.38
7,csv,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,10.68
3,csv,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,11.24




Ranking by: timeseries_polars_seconds


,dataset_format,hostname,cpu_brand,timeseries_polars_seconds
9,parquet,Inspiron3030S,Intel(R) Core(TM) i7-14700,1.93
10,parquet,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,2.22
2,csv,Inspiron3030S,Intel(R) Core(TM) i7-14700,2.40
6,csv,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,2.40
12,parquet,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,2.41
0,csv,HP-Desktop,12th Gen Intel(R) Core(TM) i9-12900,2.52
3,csv,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,2.66
13,parquet,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,2.84
7,csv,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,3.35
4,csv,MSI,AMD Ryzen AI 9 365 w/ Radeon 880M,3.49




Ranking by: timeseries_duckdb_seconds


,dataset_format,hostname,cpu_brand,timeseries_duckdb_seconds
9,parquet,Inspiron3030S,Intel(R) Core(TM) i7-14700,0.19
12,parquet,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,0.21
10,parquet,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,0.24
11,parquet,Optiplex7010,13th Gen Intel(R) Core(TM) i9-13900,0.32
13,parquet,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,0.35
6,csv,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,0.56
0,csv,HP-Desktop,12th Gen Intel(R) Core(TM) i9-12900,0.63
4,csv,MSI,AMD Ryzen AI 9 365 w/ Radeon 880M,0.64
2,csv,Inspiron3030S,Intel(R) Core(TM) i7-14700,0.67
3,csv,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,0.83


In [38]:
# Filter Group Pandas
fg_p = b_df[["hostname", "cpu_brand","dataset_size", "dataset_format", "filter_group_pandas_seconds"]]
fg_p = fg_p.groupby(by=["hostname", "cpu_brand","dataset_size", "dataset_format"]).min().reset_index()
fg_p = fg_p.rename(columns={"filter_group_pandas_seconds": "min_time_seconds"})
fg_p = fg_p.sort_values(by=["dataset_size","min_time_seconds", "hostname",  "dataset_format"])
fg_p

,hostname,cpu_brand,dataset_size,dataset_format,min_time_seconds
3,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,parquet,4.54
5,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,10000000,parquet,4.92
10,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,10000000,parquet,5.36
12,TS-P350-01,11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz,10000000,parquet,6.10
8,Optiplex7010,13th Gen Intel(R) Core(TM) i9-13900,10000000,parquet,6.30
9,StealthAI16,AMD Ryzen AI 9 365 w/ Radeon 880M,10000000,csv,10.41
2,Inspiron3030S,Intel(R) Core(TM) i7-14700,10000000,csv,10.67
4,Legion7-16IRX9,Intel(R) Core(TM) i9-14900HX,10000000,csv,10.99
0,HP-Desktop,12th Gen Intel(R) Core(TM) i9-12900,10000000,csv,12.34
7,Optiplex7010,13th Gen Intel(R) Core(TM) i9-13900,10000000,csv,13.57


In [ ]:
print(f"The fastest filter_group_pandas_seconds is {filter_group_df['filter_group_pandas_seconds'].min()} seconds")